In [1]:
from pathlib import Path

import ee
import eemont  # noqa: F401
import geemap
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from bisonlab.data import landsat_7_sr, landsat_8_sr, landsat_9_sr
from bisonlab.io import kml_to_geodataframe
from bisonlab.utils import mask_exclude

In [2]:
# ee.Authenticate()
ee.Initialize()

## Load Parcels and Masks

In [38]:
# Set path to local data directory
local_data = Path().cwd().parent.parent.parent / "data" / "local"

In [60]:
df_parcels = kml_to_geodataframe(local_data / "WRTBI-Morton Soil Types.kml")
df_parcels = df_parcels.rename(columns={"Name": "subsection", "layer": "parcel"})
df_parcels = df_parcels.drop(columns="Description")

In [61]:
df_parcels["area_m2"] = df_parcels['geometry'].to_crs('ESRI:102009').area
df_parcels["acres"] = df_parcels["area_m2"] / 4046.86

In [59]:
df_parcels.groupby('parcel').sum().round(0)

,area_m2,acres
parcel,,
Business as Usual,163349.0,40.0
Hellyer Tribal Lease,404866.0,100.0
Morton,867163.0,214.0
Shoshone Tribe,713518.0,176.0
Untouched,242528.0,60.0


In [62]:
df_parcels

,subsection,geometry,parcel,area_m2,acres
0,High Steppe Unirrigated 1,"POLYGON ((-108.79089 43.19890, -108.79081 43.1...",Shoshone Tribe,435159.384798,107.530131
1,High Steppe Unirrigated 2,"POLYGON ((-108.78826 43.20636, -108.79311 43.2...",Shoshone Tribe,54337.844709,13.427162
2,High Steppe Unirrigated 3,"POLYGON ((-108.78802 43.20623, -108.78813 43.2...",Shoshone Tribe,76528.291131,18.910536
3,High Steppe Unirrigated 4,"POLYGON ((-108.79089 43.20229, -108.78865 43.2...",Shoshone Tribe,30985.035917,7.656562
4,River Bottom 1,"POLYGON ((-108.79085 43.19374, -108.78828 43.1...",Shoshone Tribe,12504.736574,3.089985
5,River Bottom 2,"POLYGON ((-108.78594 43.19561, -108.78649 43.1...",Shoshone Tribe,70237.308702,17.356002
6,River Bottom 3,"POLYGON ((-108.79601 43.20457, -108.79447 43.2...",Shoshone Tribe,33765.226630,8.343562
7,River Bottom 1,"POLYGON ((-108.80041 43.19793, -108.80030 43.2...",Hellyer Tribal Lease,404865.920746,100.044459
8,Irrigated High Steppe 1,"POLYGON ((-108.78852 43.20122, -108.78735 43.2...",Morton,582900.146051,144.037636
9,Irrigated High Steppe 2,"POLYGON ((-108.77009 43.19749, -108.77011 43.1...",Morton,284263.109433,70.242882


In [29]:
# # Union all "mask" polygons and create a separate mask dataframe
# idx = df_parcels["parcel"] == "mask"
# df_mask = df_parcels[idx].dissolve(by="parcel").reset_index()

# # Drop "mask" from parcel dataframe
# df_parcels = df_parcels.drop(index=idx.index[idx])

# Convert parcels dataframe to ee.featureCollection
parcels = geemap.geopandas_to_ee(df_parcels)
# mask = geemap.geopandas_to_ee(df_mask)

In [43]:
m = geemap.Map()
m.addLayer(ee_parcels, {}, "")
m.centerObject(ee_parcels, 14)
m

Map(center=[43.20091661951649, -108.79407203783843], controls=(WidgetControl(options=['position', 'transparent…